In [1]:
from dataset_interface.coco import COCODataAPI
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
from imageio import imread, imwrite
import copy
import xml.etree.ElementTree
import glob
import shutil

In [2]:
data_dir = 'coco/'
config_file_val = 'coco/sample_coco_configs_val.yml'
config_file_train = 'coco/sample_coco_configs_train.yml'
coco_api_validation = COCODataAPI(data_dir, config_file_val)
coco_api_training = COCODataAPI(data_dir, config_file_train)

/home/lucy/.local/lib/python3.5/site-packages/dataset_interface-1.0-py3.5-linux-x86_64.egg/dataset_interface/common/image_data_api.py:146: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self._configurations = yaml.load(config_file)


loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
loading annotations into memory...
Done (t=14.05s)
creating index...
index created!


## Categories in dataset

In [3]:
for k, category in coco_api_training.get_sub_categories('furniture').items():
    print(k, category.name)

64 potted plant
65 bed
67 dining table
70 toilet
62 chair
63 couch


In [4]:
# List of desired coco categories: 

desired_categories_to_id = {'spoon':50, 'orange':55, 'dining table':67, 'bowl':51, \
                           'cup':47, 'fork':48, 'bed':65, 'chair':62, 'couch':63, 'knife': 49, 'person': 1}

coco_cat_id_to_name = dict()
for key,value in desired_categories_to_id.items():
    coco_cat_id_to_name[value] = key
coco_cat_id_to_name

{1: 'person',
 47: 'cup',
 48: 'fork',
 49: 'knife',
 50: 'spoon',
 51: 'bowl',
 55: 'orange',
 62: 'chair',
 63: 'couch',
 65: 'bed',
 67: 'dining table'}

In [ ]:
# coco_api.get_images_in_category(55)

## Load the images id and image object of the desired categories

In [5]:
train_images_dictionaries = {}
val_images_dictionaries = {}

print('Loading images in categories {}'.format(list(desired_categories_to_id.keys())))
for key, category_id in desired_categories_to_id.items():
    print('collecting images of {} with Id {} '.format(key,category_id))
    for image_id, image_inf in coco_api_training.get_images_in_category(category_id).items():
        if image_id in train_images_dictionaries:
            continue
        boxes = coco_api_training.get_bounding_boxes_by_ids(image_id, list(desired_categories_to_id.values()))
        train_images_dictionaries[image_id] = {'info': image_inf, 'bounding_boxes': boxes}
    
    for image_id, image_inf in coco_api_validation.get_images_in_category(category_id).items():
        if image_id in val_images_dictionaries:
            continue
        boxes = coco_api_validation.get_bounding_boxes_by_ids(image_id, list(desired_categories_to_id.values()))
        val_images_dictionaries[image_id] = {'info': image_inf, 'bounding_boxes': boxes}
    
print('Images loaded')

Loading images in categories ['knife', 'cup', 'dining table', 'fork', 'bed', 'chair', 'couch', 'bowl', 'person', 'orange', 'spoon']
collecting images of knife with Id 49 
collecting images of cup with Id 47 
collecting images of dining table with Id 67 
collecting images of fork with Id 48 
collecting images of bed with Id 65 
collecting images of chair with Id 62 
collecting images of couch with Id 63 
collecting images of bowl with Id 51 
collecting images of person with Id 1 
collecting images of orange with Id 55 
collecting images of spoon with Id 50 
Images loaded


In [ ]:
len(val_images_dictionaries)

## Remove images 

In [3]:
filenames = glob.glob('robocup_objects/training_images/*.jpg')
for fn in filenames:
    img_num = int(os.path.basename(fn).split('.')[0])
    if img_num > 15199:
        os.remove(fn)
print('done')

done


## Read the annotations file

In [4]:
val_annotations_file = 'robocup_objects/val_annotations.yml'
with open(val_annotations_file, 'r') as annotation_file:
    val_annotations = yaml.safe_load(annotation_file)
    
print('Validation annotations ', len(val_annotations))
    
train_annotations_file = 'robocup_objects/train_annotations.yml'
with open(train_annotations_file, 'r') as annotation_file:
    train_annotations = yaml.safe_load(annotation_file)

print('Training annotations ', len(train_annotations))


Validation annotations  3000
Training annotations  15200


## Get the number of the latest image generated

In [7]:
img_name = val_annotations[-1]['image_name']
latest_image_num = int(os.path.basename(img_name).replace('.jpg',''))
print(val_annotations[-1].keys())

dict_keys(['image_name', 'objects'])


## Load classes file

In [6]:
classes_file = 'classes_robocup.yml'
with open(classes_file, 'r') as classes_file:
    classes = yaml.safe_load(classes_file)

classes_to_id = dict()
for key,value in classes.items():
    classes_to_id[value] = key

## Annotation format 
* image_name 
* objects:
    - category_id 
    - xmin
    - xmax
    - ymin
    - ymax

In [13]:
# Flag 
dataset = 'train'
output_annotations = []

cat_idx = 0
desired_categories = list(desired_categories_to_id.keys())
print(desired_categories)

if dataset == 'val':
    images_dictionaries = copy.deepcopy(val_images_dictionaries)
    img_name_ = val_annotations[-1]['image_name']
    latest_image_num = int(os.path.basename(img_name_).replace('.jpg',''))
    print('Starting to add validation images at index ', latest_image_num)
else:
    images_dictionaries = copy.deepcopy(train_images_dictionaries)
    img_name_ = train_annotations[-1]['image_name']
    latest_image_num = int(os.path.basename(img_name_).replace('.jpg',''))
    print('Starting to add training images at index ', latest_image_num)

# for dict_ in images_dictionaries:
#     category = desired_categories[cat_idx]
#     print('Images in category ', category)
for image_id, image_data in images_dictionaries.items():
# shutil.copyfile
    latest_image_num += 1

    print('Adding image number {}'.format(latest_image_num))

    if dataset == 'val':
#         objects = coco_api_validation.get_bounding_boxes_by_names(id_, [category])
#         objects = objects[coco_api_validation.get_category_id(category)]
        output_path = os.path.join('robocup_objects/validation_images/',str(latest_image_num) \
             + '.jpg')
        image_name = os.path.join('validation_images',str(latest_image_num) \
             + '.jpg')
    else:
#         objects = coco_api_training.get_bounding_boxes_by_names(id_, [category])
#         objects = objects[coco_api_training.get_category_id(category)]
        output_path = os.path.join('robocup_objects/training_images/',str(latest_image_num) \
             + '.jpg')
        image_name = os.path.join('training_images',str(latest_image_num) \
             + '.jpg')

    image_inf = image_data['info']
    bounding_boxes = image_data['bounding_boxes']
    detected_objects = []

    for category_id, boxes in bounding_boxes.items():
        class_name = coco_cat_id_to_name[category_id]
        for box in boxes:
            object_dict = {'class_id': classes_to_id[class_name.replace(' ','_')],
                           'xmin': box['min_x'],
                           'xmax': box['min_x'] + box['width'],
                           'ymin': box['min_y'],
                           'ymax': box['min_y'] + box['height']}

            detected_objects.append(object_dict)
    shutil.copyfile(image_inf.image_path, output_path)
    if dataset == 'val':
        val_annotations.append({'image_name': image_name,'objects': detected_objects})
    else:
        train_annotations.append({'image_name': image_name,'objects': detected_objects})
#         
        
#     cat_idx +=1

['knife', 'cup', 'dining table', 'fork', 'bed', 'chair', 'couch', 'bowl', 'person', 'orange', 'spoon']
Starting to add training images at index  15199
Adding image number 15200
Adding image number 15201
Adding image number 15202
Adding image number 15203
Adding image number 15204
Adding image number 15205
Adding image number 15206
Adding image number 15207
Adding image number 15208
Adding image number 15209
Adding image number 15210
Adding image number 15211
Adding image number 15212
Adding image number 15213
Adding image number 15214
Adding image number 15215
Adding image number 15216
Adding image number 15217
Adding image number 15218
Adding image number 15219
Adding image number 15220
Adding image number 15221
Adding image number 15222
Adding image number 15223
Adding image number 15224
Adding image number 15225
Adding image number 15226
Adding image number 15227
Adding image number 15228
Adding image number 15229
Adding image number 15230
Adding image number 15231
Adding image numb

Adding image number 15646
Adding image number 15647
Adding image number 15648
Adding image number 15649
Adding image number 15650
Adding image number 15651
Adding image number 15652
Adding image number 15653
Adding image number 15654
Adding image number 15655
Adding image number 15656
Adding image number 15657
Adding image number 15658
Adding image number 15659
Adding image number 15660
Adding image number 15661
Adding image number 15662
Adding image number 15663
Adding image number 15664
Adding image number 15665
Adding image number 15666
Adding image number 15667
Adding image number 15668
Adding image number 15669
Adding image number 15670
Adding image number 15671
Adding image number 15672
Adding image number 15673
Adding image number 15674
Adding image number 15675
Adding image number 15676
Adding image number 15677
Adding image number 15678
Adding image number 15679
Adding image number 15680
Adding image number 15681
Adding image number 15682
Adding image number 15683
Adding image

Adding image number 16190
Adding image number 16191
Adding image number 16192
Adding image number 16193
Adding image number 16194
Adding image number 16195
Adding image number 16196
Adding image number 16197
Adding image number 16198
Adding image number 16199
Adding image number 16200
Adding image number 16201
Adding image number 16202
Adding image number 16203
Adding image number 16204
Adding image number 16205
Adding image number 16206
Adding image number 16207
Adding image number 16208
Adding image number 16209
Adding image number 16210
Adding image number 16211
Adding image number 16212
Adding image number 16213
Adding image number 16214
Adding image number 16215
Adding image number 16216
Adding image number 16217
Adding image number 16218
Adding image number 16219
Adding image number 16220
Adding image number 16221
Adding image number 16222
Adding image number 16223
Adding image number 16224
Adding image number 16225
Adding image number 16226
Adding image number 16227
Adding image

Adding image number 16727
Adding image number 16728
Adding image number 16729
Adding image number 16730
Adding image number 16731
Adding image number 16732
Adding image number 16733
Adding image number 16734
Adding image number 16735
Adding image number 16736
Adding image number 16737
Adding image number 16738
Adding image number 16739
Adding image number 16740
Adding image number 16741
Adding image number 16742
Adding image number 16743
Adding image number 16744
Adding image number 16745
Adding image number 16746
Adding image number 16747
Adding image number 16748
Adding image number 16749
Adding image number 16750
Adding image number 16751
Adding image number 16752
Adding image number 16753
Adding image number 16754
Adding image number 16755
Adding image number 16756
Adding image number 16757
Adding image number 16758
Adding image number 16759
Adding image number 16760
Adding image number 16761
Adding image number 16762
Adding image number 16763
Adding image number 16764
Adding image

Adding image number 17267
Adding image number 17268
Adding image number 17269
Adding image number 17270
Adding image number 17271
Adding image number 17272
Adding image number 17273
Adding image number 17274
Adding image number 17275
Adding image number 17276
Adding image number 17277
Adding image number 17278
Adding image number 17279
Adding image number 17280
Adding image number 17281
Adding image number 17282
Adding image number 17283
Adding image number 17284
Adding image number 17285
Adding image number 17286
Adding image number 17287
Adding image number 17288
Adding image number 17289
Adding image number 17290
Adding image number 17291
Adding image number 17292
Adding image number 17293
Adding image number 17294
Adding image number 17295
Adding image number 17296
Adding image number 17297
Adding image number 17298
Adding image number 17299
Adding image number 17300
Adding image number 17301
Adding image number 17302
Adding image number 17303
Adding image number 17304
Adding image

...

Adding image number 93612
Adding image number 93613
Adding image number 93614
Adding image number 93615
Adding image number 93616
Adding image number 93617
Adding image number 93618
Adding image number 93619
Adding image number 93620
Adding image number 93621
Adding image number 93622
Adding image number 93623
Adding image number 93624
Adding image number 93625
Adding image number 93626
Adding image number 93627
Adding image number 93628
Adding image number 93629
Adding image number 93630
Adding image number 93631
Adding image number 93632
Adding image number 93633
Adding image number 93634
Adding image number 93635
Adding image number 93636
Adding image number 93637
Adding image number 93638
Adding image number 93639
Adding image number 93640
Adding image number 93641
Adding image number 93642
Adding image number 93643
Adding image number 93644
Adding image number 93645
Adding image number 93646
Adding image number 93647
Adding image number 93648
Adding image number 93649
Adding image

Adding image number 93965
Adding image number 93966
Adding image number 93967
Adding image number 93968
Adding image number 93969
Adding image number 93970
Adding image number 93971
Adding image number 93972
Adding image number 93973
Adding image number 93974
Adding image number 93975
Adding image number 93976
Adding image number 93977
Adding image number 93978
Adding image number 93979
Adding image number 93980
Adding image number 93981
Adding image number 93982
Adding image number 93983
Adding image number 93984
Adding image number 93985
Adding image number 93986
Adding image number 93987
Adding image number 93988
Adding image number 93989
Adding image number 93990
Adding image number 93991
Adding image number 93992
Adding image number 93993
Adding image number 93994
Adding image number 93995
Adding image number 93996
Adding image number 93997
Adding image number 93998
Adding image number 93999
Adding image number 94000
Adding image number 94001
Adding image number 94002
Adding image

Adding image number 94422
Adding image number 94423
Adding image number 94424
Adding image number 94425
Adding image number 94426
Adding image number 94427
Adding image number 94428
Adding image number 94429
Adding image number 94430
Adding image number 94431
Adding image number 94432
Adding image number 94433
Adding image number 94434
Adding image number 94435
Adding image number 94436
Adding image number 94437
Adding image number 94438
Adding image number 94439
Adding image number 94440
Adding image number 94441
Adding image number 94442
Adding image number 94443
Adding image number 94444
Adding image number 94445
Adding image number 94446
Adding image number 94447
Adding image number 94448
Adding image number 94449
Adding image number 94450
Adding image number 94451
Adding image number 94452
Adding image number 94453
Adding image number 94454
Adding image number 94455
Adding image number 94456
Adding image number 94457
Adding image number 94458
Adding image number 94459
Adding image

Adding image number 94824
Adding image number 94825
Adding image number 94826
Adding image number 94827
Adding image number 94828
Adding image number 94829
Adding image number 94830
Adding image number 94831
Adding image number 94832
Adding image number 94833
Adding image number 94834
Adding image number 94835
Adding image number 94836
Adding image number 94837
Adding image number 94838
Adding image number 94839
Adding image number 94840
Adding image number 94841
Adding image number 94842
Adding image number 94843
Adding image number 94844
Adding image number 94845
Adding image number 94846
Adding image number 94847
Adding image number 94848
Adding image number 94849
Adding image number 94850
Adding image number 94851
Adding image number 94852
Adding image number 94853
Adding image number 94854
Adding image number 94855
Adding image number 94856
Adding image number 94857
Adding image number 94858
Adding image number 94859
Adding image number 94860
Adding image number 94861
Adding image

Adding image number 95168
Adding image number 95169
Adding image number 95170
Adding image number 95171
Adding image number 95172
Adding image number 95173
Adding image number 95174
Adding image number 95175
Adding image number 95176
Adding image number 95177
Adding image number 95178
Adding image number 95179
Adding image number 95180
Adding image number 95181
Adding image number 95182
Adding image number 95183
Adding image number 95184
Adding image number 95185
Adding image number 95186
Adding image number 95187
Adding image number 95188
Adding image number 95189
Adding image number 95190
Adding image number 95191
Adding image number 95192
Adding image number 95193
Adding image number 95194
Adding image number 95195
Adding image number 95196
Adding image number 95197
Adding image number 95198
Adding image number 95199
Adding image number 95200
Adding image number 95201
Adding image number 95202
Adding image number 95203
Adding image number 95204
Adding image number 95205
Adding image

## Save annotations

In [8]:
dataset = 'train'
if dataset == 'val':
    annotations_file = 'robocup_objects/val_annotations.yml'
    output_annotations = val_annotations
else:
    print('saving training annotations')
    annotations_file = 'robocup_objects/train_annotations.yml'
    output_annotations = train_annotations

with open(annotations_file, 'w') as annotation_file:
    yaml.safe_dump(output_annotations, annotation_file,default_flow_style=False,
                   encoding='utf-8')
    
print('done saving')

saving training annotations
done saving


In [16]:
train_annotations = train_annotations[0: 95496]

len(train_annotations)

95496

# Manual labeled images

In [7]:
trash_can_path = 'trash_can/'
xml_files = glob.glob(trash_can_path+'/*.xml')
img_files = glob.glob(trash_can_path+'/*.jpg')
# print(img_files)

img_name_ = train_annotations[-1]['image_name']
latest_image_num = int(os.path.basename(img_name_).replace('.jpg',''))

for idx, xml_file in enumerate(xml_files):
    latest_image_num += 1
    root = xml.etree.ElementTree.parse(xml_file).getroot()
    
    image_name = os.path.join('training_images',str(latest_image_num) + '.jpg') 
    file_name = root.find('filename').text
    output_path = os.path.join('robocup_objects/training_images/',str(latest_image_num) \
                 + '.jpg')
    
#     print(file_name)
    
    object_dict = {'class_id': classes_to_id['trash_can'],
                   'xmin': int(root.find('object').find('bndbox').find('xmin').text),
                   'xmax': int(root.find('object').find('bndbox').find('xmax').text),
                   'ymin': int(root.find('object').find('bndbox').find('ymin').text),
                   'ymax': int(root.find('object').find('bndbox').find('ymax').text)}
    
    train_annotations.append({'image_name': image_name,'objects': [object_dict]})
    
    img = imread(img_files[idx]) 
    imwrite(output_path, img)
    
print(len(train_annotations))

15216
